In [1]:
import sext_tools
import pandas as pd

### Notebook to compute and verify null space knobs

In [2]:
# Load lattice
sdds = sext_tools.get_lattice_beamsheet()
df = sdds.columns_to_df()
df

,ElementName,ElementParameter,ParameterValue,ParameterValueString,ElementType,ElementOccurence,ElementGroup
0,MK4G1C30A,DX,0.000000,,MARK,1,
1,MK4G1C30A,DY,0.000000,,MARK,1,
2,MK4G1C30A,FITPOINT,0.000000,,MARK,1,
3,ISSP1D,L,0.799513,,KICKER,1,
4,ISSP1D,HKICK,0.000000,,KICKER,1,
...,...,...,...,...,...,...,...
52446,ISSP1U,ORDER,0.000000,,KICKER,1,
52447,ISSP1U,STEERING,1.000000,,KICKER,1,
52448,ISSP1U,SYNCH_RAD,0.000000,,KICKER,1,
52449,ISSP1U,ISR,0.000000,,KICKER,1,


In [3]:
sextupoles = df[(df.ElementType == 'KSEXT') &(df.ElementParameter == 'K2')]
sextupoles

,ElementName,ElementParameter,ParameterValue,ParameterValueString,ElementType,ElementOccurence,ElementGroup
100,SH1G2C30A,K2,19.832912,,KSEXT,1,
449,SH3G2C30A,K2,-5.855108,,KSEXT,1,
565,SH4G2C30A,K2,-15.820901,,KSEXT,1,
871,SM1G4C30A,K2,-23.680634,,KSEXT,1,
1006,SM2G4C30B,K2,28.643155,,KSEXT,1,
...,...,...,...,...,...,...,...
51596,SM2G4C29B,K2,28.643155,,KSEXT,1,
51696,SM1G4C29B,K2,-25.946035,,KSEXT,1,
51990,SH4G6C29B,K2,-15.820901,,KSEXT,1,
52106,SH3G6C29B,K2,-5.855108,,KSEXT,1,


In [4]:
# Get groups based on preset
MODE = '6and3'
groups, groups_direct, indices = sext_tools.make_knobs(mode=MODE)
all_groups = {**groups, **groups_direct}
groups['SM1A'][:5]

['SM1G4C01A', 'SM1G4C02A', 'SM1G4C03A', 'SM1G4C04A', 'SM1G4C05A']

In [5]:
# Confirm ALL sextupoles are in groups
s1 = set(sextupoles.ElementName)
s2 = set([y for x in all_groups.values() for y in x])
print(len(s1), len(s2))
assert s1 == s2, f'{s1-s2}'

270 270


In [6]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
dfknobs = sext_tools.calculate_null_knobs('6and3')
dfknobs

File lattice\nullknobs.param will be overwritten


,SVD0,SH1,SH3,SH4,SL1,SL2,SL3
SL1,0.000000,0.0,0.0,0.0,1.0,0.0,0.0
SL2,0.000000,0.0,0.0,0.0,0.0,1.0,0.0
SL3,0.000000,0.0,0.0,0.0,0.0,0.0,1.0
SH1,0.000000,1.0,0.0,0.0,0.0,0.0,0.0
SH3,0.000000,0.0,1.0,0.0,0.0,0.0,0.0
SH4,0.000000,0.0,0.0,1.0,0.0,0.0,0.0
SM1A,-0.663666,0.0,0.0,0.0,0.0,0.0,0.0
SM1B,0.744816,0.0,0.0,0.0,0.0,0.0,0.0
SM2B,-0.069260,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Final mapping of [element] -> [change]
null_knobs = sext_tools.make_knob_groups(all_groups, dfknobs)
null_knobs['SVD0']['SM1G4C01A']

-0.663665593142482

In [8]:
# Apply 10*knob as test
kb = {k:v*10 for k,v in null_knobs['SH3'].items()}
print(sext_tools.get_chroma(kb))

File lattice\nullknobs.param will be overwritten


{'dnux/dp': [2.0000027539178844, 1.9808773479518844], 'dnuy/dp': [1.789887420705868, 1.8094444074949896]}


In [15]:
dfknobs.to_json(f'lattice/dfknobs_{MODE}.json')
data_dict = {'groups': groups,
             'groups_direct': groups_direct,
             'indices': indices,
             'null_knobs': null_knobs}
import json
with open(f'lattice/data_{MODE}.json', 'w') as f:
    json.dump(data_dict, f)